In [ ]:
import pandas as pd
import numpy as np
import pyfolio as pf
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import pickle
import yaml
use_bbg = True
if use_bbg:
    from bbgclient import SyncClient
    import pdblp
    con = pdblp.BCon(timeout=500000, debug = False)
    con.start()
    bbg = SyncClient(False, False)
from datetime import date 
plt.rcParams['figure.figsize'] = 14, 10
plt.rcParams['figure.figsize'] = 14, 10
import warnings
warnings.filterwarnings("ignore")

In [8]:
def roll_ticker(bbg_ticker,roll_type, adjust_type, days_exp):
    dict_adjust = {'None': 'N', 'Difference':'D', 'Ratio': 'R', 'Average': 'W'}
    if len(roll_type) == 1:
        lst = bbg_ticker.split()
        bbg_ticker_aux = ' '.join([w for w in lst if w != lst[-1]])
        if days_exp<10:
            return bbg_ticker_aux + ' ' + roll_type + ':' + '0'+ str(days_exp) + '_' + '0' + '_' +dict_adjust[adjust_type] +' '+ lst[-1]
        else:
            return bbg_ticker_aux + ' ' + roll_type + ':' + str(days_exp) + '_' + '0' + '_' +dict_adjust[adjust_type] +' '+ lst[-1]
    else:
        dict_rolls = {'Bloomberg Default':'B', 
                'Relative to Expiration' : 'R',
                'Fixed Day of Month': 'F',
                'With Active Future': 'A',
                'Relative to First Notice': 'N',
                'At First Delivery': 'D',
                'At option expiration':'O'}        
        lst = bbg_ticker.split()
        bbg_ticker_aux = ' '.join([w for w in lst if w != lst[-1]])
        if days_exp<10:
            return bbg_ticker_aux + ' ' + dict_rolls[roll_type] + ':' + '0'+ str(days_exp) + '_' + '0' +'_' + dict_adjust[adjust_type] +' '+ lst[-1]
        else:
            return bbg_ticker_aux + ' ' + dict_rolls[roll_type] + ':' + str(days_exp) + '_' + '0' +'_' + dict_adjust[adjust_type] +' '+ lst[-1]

def bbg_data(list_data, list_fields, start_date, end_date):
    data_bbg =  con.bdh(list_data, 
                        list_fields, 
                        start_date, 
                        end_date)
#                         elms=[("calendarCodeOverride", "5D")])
    return data_bbg

def bbg_to_df(df_bbg, dict_aux, cumreturn=True):
    columns_df = [dict_aux[i[0]] for i in df_bbg.columns]
    if cumreturn:
        df_return = ((pd.DataFrame(df_bbg.values, index = df_bbg.index, columns = columns_df)).pct_change().fillna(0) + 1).cumprod()
        return df_return
    else:
        df_ = pd.DataFrame(df_bbg.values, index = df_bbg.index, columns = columns_df)
        return df_    

In [9]:
all_assets = pd.read_excel('assets.xlsx', engine = 'openpyxl')
futures = all_assets.dropna()
futures_tickers = []
for f in futures.index:
    futures_tickers.append(roll_ticker(futures.loc[f].Ticker, futures.loc[f].RollType, futures.loc[f].AdjustType, int(futures.loc[f].Days_exp)))
otc = all_assets[~all_assets.index.isin(futures.index)].dropna(axis=1)
otc_tickers = otc.Ticker.tolist()

dict_aux_fut = {x:y for x,y in zip(futures_tickers, futures.Name)}
dict_aux_oct = {x:y for x,y in zip(otc_tickers, [i.split()[0] for i in otc_tickers])}

date_parser = lambda x: x.strftime('%Y%m%d')
start_date = "19870102"
# start_date = "20190402"
today = date.today()
end_date = date_parser(today)

In [10]:
futures_data = bbg_to_df(bbg_data(futures_tickers, ['CONTRACT_VALUE'], start_date, end_date), dict_aux_fut, cumreturn=False).fillna(method='ffill')
oct_data = bbg_to_df(bbg_data(otc_tickers, ['PX_LAST'], start_date, end_date), dict_aux_oct, cumreturn=False).fillna(method='ffill')

In [11]:
futures_data.tail()

,FTSE,ESTOXX50,GOLD,CAD,KOSPI,DAX,NATUGAS,SILVER,BRL,EUR,...,BA_2,BA_3,US2Y,WHEATKANSAS,FF_2,BA_4,US5Y,SUGARICE,FF_3,FF_4
date,,,,,,,,,,,,,,,,,,,,,
2021-02-17,452550.0,36970.0,177280.0,78750.0,105862500.0,347675.0,30320.0,136775.0,270965.30,150537.50,...,248850.00,248825.0,221000.00,31512.5,416387.47,248775.00,125406.25,23175.0,416366.64,416366.64
2021-02-18,445900.0,36750.0,177500.0,78865.0,104337500.0,347075.0,29700.0,135600.0,271844.40,151143.75,...,248837.50,248812.5,221007.81,32150.0,416387.47,248762.50,125476.56,23660.0,416366.64,416366.64
2021-02-19,448950.0,37140.0,177740.0,79235.0,105362500.0,350075.0,29910.0,136475.0,270442.25,151518.75,...,248837.50,248812.5,221015.63,31912.5,416408.31,248750.00,125351.56,24070.0,416366.64,416366.64
2021-02-22,449350.0,37000.0,180840.0,79370.0,104225000.0,349000.0,29360.0,140650.0,272020.40,152162.50,...,248837.50,248812.5,220992.19,32550.0,416387.47,248750.00,125273.44,24420.0,416345.81,416324.97
2021-02-23,454450.0,36820.0,180270.0,79345.0,104075000.0,346075.0,28760.0,138350.0,271325.00,151943.75,...,248837.51,248825.0,220992.19,32375.0,416387.49,248762.49,125359.38,23920.0,416345.81,416324.99


In [21]:
list(futures_data.columns)


['FTSE',
 'ESTOXX50',
 'GOLD',
 'CAD',
 'KOSPI',
 'DAX',
 'NATUGAS',
 'SILVER',
 'BRL',
 'EUR',
 'CHF',
 'CORN',
 'PLATINUM',
 'OMX',
 'USD',
 'WHEAT',
 'PALLADIUM',
 'GBP',
 'CAC40',
 'JPY',
 'COPPER',
 'RUB',
 'SOYBEAN',
 'FTSEMIB',
 'SGX',
 'ASXPI200',
 'AUD',
 'INR',
 'COFFE',
 'FTSE_JSE',
 'SPTSX60',
 'NICKEL',
 'NZD',
 'RTS',
 'MXN',
 'COCOA',
 'FTSE100',
 'NASDAQ',
 'ALUMINUM',
 'KRW',
 'SUGAR',
 'IBOV',
 'CNH',
 'NIKKEI',
 'ZAR',
 'COTTON',
 'CRUDE',
 'SP500',
 'US10Y',
 'SOYBEANOIL',
 'BRENT',
 'JP10Y',
 'S_1',
 'ER_1',
 'DE10Y',
 'ER_2',
 'GB10Y',
 'S_2',
 'DE2Y',
 'HOG',
 'ER_3',
 'SOYBEANMEAL',
 'S_3',
 'KRW3Y',
 'ER_4',
 'DE5Y',
 'FR10Y',
 'IT3Y',
 'S_4',
 'CAD10Y',
 'CATTLE',
 'ES_1',
 'AU10Y',
 'A_1',
 'ED_1',
 'LEAD',
 'GASOLINE',
 'ES_2',
 'AU3Y',
 'A_2',
 'ED_2',
 'ES_3',
 'A_3',
 'ZB_1',
 'ES_4',
 'ED_3',
 'SFR_1',
 'ZINC',
 'SFR_2',
 'A_4',
 'SFR_3',
 'ZB_2',
 'SFR_4',
 'GASOIL',
 'ED_4',
 'ZB_3',
 'ZB_4',
 'HEATINGOIL',
 'FEEDER',
 'FF_1',
 'BA_1',
 'US10Y',
 'BA_2

In [26]:
futures_data.tail(20)

,FTSE,ESTOXX50,GOLD,CAD,KOSPI,DAX,NATUGAS,SILVER,BRL,EUR,...,BA_2,BA_3,US2Y,WHEATKANSAS,FF_2,BA_4,US5Y,SUGARICE,FF_3,FF_4
date,,,,,,,,,,,,,,,,,,,,,
2021-01-27,443750.0,35320.0,184890.0,78170.0,105737500.0,340850.0,26690.0,127150.0,269688.25,151406.25,...,248875.00,248862.5,220984.38,32125.0,416366.64,248825.00,126023.44,21220.0,416387.47,416408.31
2021-01-28,444850.0,35480.0,184120.0,78165.0,103512500.0,341275.0,26310.0,129825.0,271973.20,151793.75,...,248875.00,248862.5,220960.94,31587.5,416366.64,248812.50,125929.69,21085.0,416387.47,416408.31
2021-01-29,430800.0,34780.0,185030.0,78120.0,101112500.0,336050.0,25320.0,134775.0,274040.85,151787.50,...,248887.50,248875.0,220976.56,32187.5,416387.48,248825.00,125875.00,21775.0,416387.47,416408.31
2021-02-01,430800.0,35250.0,186390.0,77875.0,103825000.0,340700.0,28150.0,147325.0,273212.95,150975.00,...,248887.50,248875.0,221000.00,31550.0,416387.48,248825.00,125968.75,22190.0,416387.47,416408.31
2021-02-02,440850.0,35810.0,183340.0,78100.0,105525000.0,345625.0,28100.0,132225.0,267973.90,150381.25,...,248887.50,248875.0,220976.56,31225.0,416387.48,248825.00,125875.00,22135.0,416408.31,416408.31
2021-02-03,440900.0,35970.0,183510.0,78245.0,106325000.0,347900.0,27550.0,134650.0,268885.90,150450.00,...,248875.00,248862.5,220968.75,31562.5,416387.48,248812.50,125804.69,22085.0,416408.31,416408.31
2021-02-04,442350.0,36330.0,179120.0,77940.0,104262500.0,351100.0,28990.0,131375.0,272368.25,149693.75,...,248875.00,248862.5,220968.75,31212.5,416408.31,248812.50,125820.31,22220.0,416408.31,416408.31
2021-02-05,442950.0,36470.0,181300.0,78325.0,105625000.0,351025.0,28280.0,135300.0,268458.40,150656.25,...,248875.00,248862.5,221007.81,31550.0,416408.31,248812.50,125796.88,22670.0,416387.47,416387.47
2021-02-08,443200.0,36680.0,183420.0,78490.0,104625000.0,351875.0,28460.0,138100.0,268580.55,150806.25,...,248875.00,248850.0,220984.38,32262.5,416408.31,248800.00,125742.19,22600.0,416387.47,416387.47


In [25]:
futures_data.to_csv('futures.csv')